In [2]:
import sqlite3
import requests
from bs4 import BeautifulSoup

In [28]:
r=requests.get('https://realpython.github.io/fake-jobs')
soup=BeautifulSoup(r.text, 'html.parser')


In [33]:
classes=soup.find_all(class_='card-content') 
dic={}
for idx, c in enumerate(classes):
    job_title=c.find('h2').text
    company_name=c.find('h3').text
    location=c.find(class_='location').text.strip()
    link=c.find_all('a')[1]['href']
    r=requests.get(link)
    s=BeautifulSoup(r.content,'html.parser')
    description=s.find(class_='content').find('p').text
    dic[idx]={
        'job_title': job_title,
        'company_name':company_name,
        'location': location,
        'description':description,
        'link':link
    }
dic

    

{0: {'job_title': 'Senior Python Developer',
  'company_name': 'Payne, Roberts and Davis',
  'location': 'Stewartbury, AA',
  'description': 'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.',
  'link': 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'},
 1: {'job_title': 'Energy engineer',
  'company_name': 'Vasquez-Davidson',
  'location': 'Christopherville, AA',
  'description': 'Party prevent live. Quickly candidate change although. T

In [18]:
#creating database and adding the elements scraped from webpage
con=sqlite3.connect('jobs.db')
cursor=con.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS jobs(
               'Job Title' TEXT,
               'Company Name' TEXT,
               'Location' TEXT,
               'Job Description' TEXT,
               'Application Link' TEXT,
               UNIQUE('Job Title', 'Company Name', 'Location'))'''
)
con.commit()
cursor.execute("SELECT 'Job Title', 'Company Name', 'Location' FROM jobs")
existing_jobs={f"{row[0]}|{row[1]}|{row[2]}" for row in cursor.fetchall()} # created identifiers for already existing data


for job in dic.values():
    identifier=f"{job['job_title']}|{job['company_name']}|{job['location']}"
    if identifier not in existing_jobs:
        cursor.execute("INSERT OR IGNORE INTO jobs VALUES(?,?,?,?,?)",(job['job_title'], job['company_name'], job['location'],job['description'], job['link']))
con.commit()
con.close()
print("Added")

Added


In [15]:
from datetime import datetime
con=sqlite3.connect('jobs.db')
cursor=con.cursor()
cursor.execute("SELECT 'Job Title', 'Company Name', 'Location', 'Job Description','Application Link' FROM jobs")
existing_jobs = {
    f"{row[0]}|{row[1]}|{row[2]}": {"description": row[3], "application_link": row[4]}
    for row in cursor.fetchall()
}


#scraping updated data and update records
r=requests.get('https://realpython.github.io/fake-jobs')
soup=BeautifulSoup(r.text, 'html.parser')

classes=soup.find_all(class_='card-content') 
dic={}
for idx, c in enumerate(classes):
    job_title=c.find('h2').text
    company_name=c.find('h3').text
    location=c.find(class_='location').text.strip()
    link=c.find_all('a')[1]['href']
    r=requests.get(link)
    s=BeautifulSoup(r.content,'html.parser')
    description=s.find(class_='content').find('p').text
    dic[idx]={
        'job_title': job_title,
        'company_name':company_name,
        'location': location,
        'description':description,
        'link':link
    }

    #checking if there is a change
    for job in dic.values():
        identifier = f"{job['job_title']}|{job['company_name']}|{job['location']}"
        if identifier in existing_jobs:
            existing = existing_jobs[identifier]
            # Check for updates
            if job['Description'] != existing['description'] or job['Application Link'] != existing['application_link']:
            # Update record
                cursor.execute('''
                UPDATE jobs
                SET 'Job Description' = ?, 'Application Link' = ?
                WHERE 'Job Title' = ? AND 'Company Name' = ? AND 'Location' = ?
            ''', (job['description'], job['link'],
                  job['job_title'], job['company_name'], job['location']))
                print(f"Updated job: {job['job_title']}")

# Commit changes and close the connection
con.commit()
con.close()
        

In [20]:
import csv

con=sqlite3.connect('jobs.db')
cursor=con.cursor()
def filter_jobs(location=None, company_name=None):
    query="SELECT [Job Title], [Company Name], [Location], [Job Description], [Application Link] FROM jobs WHERE 1=1"
    params=[]
    if location:
        query+=" AND [Location]=?"
        params.append(location)
    if company_name:
        query+=" AND [Company Name]=?"
        params.append(company_name)
    cursor.execute(query, params)
    return cursor.fetchall()
def export_to_csv(job_list, filename='filtered_jobs.csv'):
    headers=['Job Title', 'Company Name', 'Location', 'Job Description', 'Application Link']
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer=csv.writer(file)
        writer.writerow(headers)
        writer.writerows(job_list)
    print(f"Exported {len(job_list)} job(s) to {filename}")
filtered_jobs=filter_jobs(location='Port Ericaburgh, AA')
export=export_to_csv(filtered_jobs, 'filtered_jobs.csv')
con.close()

Exported 1 job(s) to filtered_jobs.csv
